In [3]:
import numpy as np
from matplotlib import pyplot

In [34]:
def distance(x1, y1, x2, y2):
    np.sqrt((x2 - x1)**2 + (y2 - y1)**2)

In [ ]:
Nx = 400 # How many cells in x axis
Ny = 100 # How many cells in y axis

tau = 0.53 # Time scale
Nt = 3000 # TIme steps

# lattice speeds and weights
NL = 9 # 9 differentt velocities
csx = np.array([0, 0, 1, 1, 1, 0, -1, -1, -1])
cys = np.array([0, 1, 1, 0, -1, -1, -1, 0, 1])
weights = np.array([4/9 , 1/9, 1/36, 1/9, 1/36, 1/9, 1/36, 1/9, 1/36])

# Initial Condition
F = np.ones((Ny, Nx, NL)) + (0.1 * np.random.randn(Ny, Nx, NL))
F[:, :, 3] = 2.3 # Every right node gets a nudge to the right 

cylinder = np.full((Ny, Nx), False) # False = NOT a Boundary, True = IS a Boundary
cylinder_radius = 13

# Defining Obsticle
for x in range(Nx):
    for y in range(Ny):
        if distance(Nx//4, Ny//2, x, y) <= cylinder_radius:
            cylinder[x][y] = True 

In [40]:
np.array([[1, 2, 3], [4, 5, 6], [7, 8, 9]])

array([[1, 2, 3],
       [4, 5, 6],
       [7, 8, 9]])

In [45]:
np.roll(np.array([[1, 2, 3], [4, 5, 6], [7, 8, 9]]), 1, axis = 1)

array([[3, 1, 2],
       [6, 4, 5],
       [9, 7, 8]])

In [33]:
np.full((2,3,4), 2).shape

(2, 3, 4)